In [ ]:
import torch

print(torch.cuda.is_available())

True


In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!nvidea-smi

/bin/bash: line 1: nvidea-smi: command not found


In [ ]:
! df -kH

Filesystem      Size  Used Avail Use% Mounted on
overlay          84G   30G   55G  36% /
tmpfs            68M     0   68M   0% /dev
shm             6.2G     0  6.2G   0% /dev/shm
/dev/root       2.1G  1.3G  860M  59% /usr/sbin/docker-init
tmpfs           6.9G   70k  6.9G   1% /var/colab
/dev/sda1       113G   88G   26G  78% /kaggle/input
tmpfs           6.9G     0  6.9G   0% /proc/acpi
tmpfs           6.9G     0  6.9G   0% /proc/scsi
tmpfs           6.9G     0  6.9G   0% /sys/firmware


In [ ]:
!ls -al /tmp

total 44
drwxrwxrwt 1 root root 4096 Jun 25 20:48 .
drwxr-xr-x 1 root root 4096 Jun 25 20:46 ..
-rw-r--r-- 1 root root 1217 Jun 25 20:46 dap_multiplexer.63b646b22434.root.log.INFO.20240625-204606.110
lrwxrwxrwx 1 root root   62 Jun 25 20:46 dap_multiplexer.INFO -> dap_multiplexer.63b646b22434.root.log.INFO.20240625-204606.110
srwxr-xr-x 1 root root    0 Jun 25 20:46 debugger_10abk3idas
drwx------ 2 root root 4096 Jun 25 20:46 initgoogle_syslog_dir.0
-rw-r--r-- 1 root root 2023 Jun 25 20:49 language_service.63b646b22434.root.log.INFO.20240625-204805.645
lrwxrwxrwx 1 root root   63 Jun 25 20:48 language_service.INFO -> language_service.63b646b22434.root.log.INFO.20240625-204805.645
drwx------ 2 root root 4096 Jun 25 20:48 pyright-652-4hwlbYOGPhvJ
drwx------ 2 root root 4096 Jun 25 20:48 pyright-652-WRt3kEJEfMLS
drwxr-xr-x 3 root root 4096 Jun 25 20:48 python-languageserver-cancellation
drwx------ 2 root root 4096 Jun 25 20:46 tmp30sexwp5


In [ ]:
import os
os.environ['HF_HOME'] = '/tmp'

In [ ]:
!pip install -U transformers accelerate bitsandbytes

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-25 20:58:58--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-25 20:58:58 (65.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [ ]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# def llm(prompt):
#     input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
#     outputs = model.generate(input_ids, )
#     result = tokenizer.decode(outputs[0])
#     return result

In [ ]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        # top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
%%time
rag("I just discovered the course. Can I still join it?")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)